In [1]:
import os
import re
import functools
from collections import OrderedDict
import doctest
from llparser import *
os.chdir('../form/sources')
os.getcwd()

'/home/feelus/tmp/form/sources'

In [2]:
os.listdir()

['grccparam.h',
 'Makefile.am',
 'sort.c',
 'polygcd.h',
 'compiler.c',
 'names.c',
 'if.c',
 'transform.c',
 'evaluate.c',
 'sch.c',
 'comtool.c',
 'lus.c',
 'gentopo.cc',
 'poly.h',
 'proces.c',
 'token.c',
 'normal.c',
 'mallocprotect.h',
 'declare.h',
 'fsizes.h',
 'symmetr.c',
 'inivar.h',
 'parallel.c',
 'tools.c',
 'minos.h',
 'gentopo.h',
 'wildcard.c',
 'startup.c',
 'pattern.c',
 'minos.c',
 'pre.c',
 'function.c',
 'reshuf.c',
 'parallel.h',
 'polyfact.h',
 'grcc.cc',
 'message.c',
 'diagrams.c',
 'setfile.c',
 'float.c',
 'comtool.h',
 'diawrap.cc',
 'threads.c',
 'notation.c',
 'smart.c',
 'mpidbg.h',
 'grcc.h',
 'spectator.c',
 'ratio.c',
 'topowrap.cc',
 'mytime.cc',
 'polygcd.cc',
 'ftypes.h',
 'findpat.c',
 'mpi.c',
 'dollar.c',
 'opera.c',
 'bugtool.c',
 'unixfile.c',
 'tables.c',
 'unix.h',
 'vector.h',
 'polywrap.cc',
 'form3.h',
 'dict.c',
 'structs.h',
 'model.c',
 'extcmd.c',
 'module.c',
 'execute.c',
 'argument.c',
 'optimize.cc',
 'factor.c',
 'compcomm.c',
 '

In [3]:
file1 = open('form3.h', 'r')
Lines = file1.readlines()
Lines

['/** @file form3.h\n',
 ' *\n',
 ' *  Contains critical defines for the compilation process\n',
 ' *\tAlso contains the inclusion of all necessary header files.\n',
 ' *\tThere are also some system dependencies concerning file functions.\n',
 ' */\n',
 '\n',
 '/* #[ License : */\n',
 '/*\n',
 ' *   Copyright (C) 1984-2023 J.A.M. Vermaseren\n',
 ' *   When using this file you are requested to refer to the publication\n',
 ' *   J.A.M.Vermaseren "New features of FORM" math-ph/0010025\n',
 ' *   This is considered a matter of courtesy as the development was paid\n',
 ' *   for by FOM the Dutch physics granting agency and we would like to\n',
 ' *   be able to track its scientific use to convince FOM of its value\n',
 ' *   for the community.\n',
 ' *\n',
 ' *   This file is part of FORM.\n',
 ' *\n',
 ' *   FORM is free software: you can redistribute it and/or modify it under the\n',
 ' *   terms of the GNU General Public License as published by the Free Software\n',
 ' *   Foundation, e

In [4]:
Lines[0]

'/** @file form3.h\n'

In [5]:
def strip_baks_comment(lines):
    for l in lines:
        r = ''
        p=0
        while (q:=l[p:].find('/*$')) !=-1:
            r+=l[p:q]
            p = l[q+3].index('$*/')
        q = l[p:].find('//$') if '//$'in l else -1
        r+=l[p:q]
        yield r

In [6]:
def end_merge(lines):
    r=''
    sep_positions = []
    for l in lines:
        if not l.endswith('\\'):
            yield (r+l,sep_positions)
            r=''
            sep_positions=[]
        else:
            r+=l.removesuffix('\\')
            sep_positions.append(sep_positions[-1]+len(l)-1 if len(sep_positions) else len(l)-1)
    if len(sep_positions):
        raise ValueError('aaa')

In [7]:
for x,sp in end_merge(strip_baks_comment(Lines)):
    if len(sp):
        print(x)

#define STATIC_ASSERT__3(X,L) 	typedef char static_assertion_failed_##L[(!!(X))*2-1]
#define PADDUMMY(type, size) 	UBYTE d_u_m_m_y[alignof(type) - ((size) & (alignof(type) - 1))]
#define PADPOSITION(ptr_,long_,int_,word_,byte_) 	PADDUMMY(off_t, 		+ sizeof(int *) * (ptr_) 		+ sizeof(LONG)  * (long_) 		+ sizeof(int)   * (int_) 		+ sizeof(WORD)  * (word_) 		+ sizeof(UBYTE) * (byte_) 	)
#define PADPOINTER(long_,int_,word_,byte_) 	PADDUMMY(int *, 		+ sizeof(LONG)  * (long_) 		+ sizeof(int)   * (int_) 		+ sizeof(WORD)  * (word_) 		+ sizeof(UBYTE) * (byte_) 	)
#define PADLONG(int_,word_,byte_) 	PADDUMMY(LONG, 		+ sizeof(int)   * (int_) 		+ sizeof(WORD)  * (word_) 		+ sizeof(UBYTE) * (byte_) 	)
#define PADINT(word_,byte_) 	PADDUMMY(int, 		+ sizeof(WORD)  * (word_) 		+ sizeof(UBYTE) * (byte_) 	)
#define PADWORD(byte_) 	PADDUMMY(WORD, 		+ sizeof(UBYTE) * (byte_) 	)
#define Utruncate(f) { 	if ( ftruncate(f->descriptor, 0) ) { 		MLOCK(ErrorMessageLock); 		MesPrint("Utruncate failed"); 		MUNLOCK(Er

In [8]:
def highlight_comments(lines):
    for l,sp in lines:
        pass

In [9]:
class Err: # short error
    def __init__(self,*args):
        self.x=args
    def __repr__(self):
        return 'Err'+repr(tuple(self.x))
Err(2,3)

Err(2, 3)

In [10]:
read_sequence = read_sequential
def printret(*args):
    print(*[repr(x) for x in args])
    return args[0]

def ptest(patt,string):
    global USE_RE
    tmp = USE_RE
    try:
        USE_RE = True
        rt= read_sequence(string+' ',mkpos(0), a=patt,b=' ',proc=lambda d : d.a)
        USE_RE = False
        rf= read_sequence(string+' ',mkpos(0), a=patt,b=' ',proc=lambda d : d.a)
    finally:
        USE_RE = tmp
    if isok(rt) and isok(rf):
        if rt==rf:
            return rt
        else:
            return mkdict(true=rt,false=rf)
    elif not isok(rt) and not isok(rf):
        if rt.what[-2]==rf.what[-2]:
            return Err(*rt.what[-2])
        else:
            return mkdict(true=rt,false=rf)
    return mkdict(true=rt,false=rf)

lcat = lambda l : ''.join(l)
dcat = lambda d : ''.join(v for k,v in d.items())
def lcatf(proc):
    return lambda l : proc(''.join(l))
def dcatf(proc):
    return lambda d : proc(''.join(v for k,v in d.items()))
inthex = lambda x : int(x,16)
intoct = lambda x : int(x,8)
def dflt(v):
    return lambda x : x[0] if len(x)==1 else v

optional= lambda patt,proc=None,errproc=None : repeatedly(0,1,patt,proc,errproc)
opt_des = lambda patt : optional(patt,proc=dflt('')) # default empty string
rep_cat = lambda min,max,patt : repeatedly(min,max,patt, proc=lcat)
seq_cat = lambda **kvargs : sequential(**kvargs,proc=dcat)

def select_longest(lp):
    rr=None
    pp=-1
    for r,p in lp:
        if p>pp:
            pp=p
            rr=r
        elif p==pp:
            raise ValueError('ambiguous results with same length',r,p,rr,pp)
    return (rr,pp)
def compose(*funs):
    def fun(r):
        for f in reversed(funs):
            r=f(r)
        return r
    return fun
def dict_append(d,**kvargs):
    for k,v in kvargs.items():
        d[k]=v
    return d
def dict_delete(d,**kvargs):
    for k,v in kvargs.items():
        del d[k]
    return d


In [11]:
USE_RE = True

punctuator        = char_in_set('[](){}<>.,+-&*/~!%=^|?:;#',errproc='punctuator')
digit             = char_in_set('0123456789'               ,errproc='digit')
hexadecimal_digit = char_in_set('0123456789abcdefABCDEF'   ,errproc='hexadecimal digit')
octal_digit       = char_in_set('01234567'                 ,errproc='octal digit')
nondigit          = char_in_set('_'+''.join(chr(i) for i in range(ord('a'),ord('z')+1))+\
                                    ''.join(chr(i) for i in range(ord('A'),ord('Z')+1)),errproc='nondigit')

@cacheread
def hexadecimal_prefix(s,pos):
    '''
    >>> ptest(hexadecimal_prefix,'0x')
    '0x'
    >>> ptest(hexadecimal_prefix,'0000')
    Err(0, 'hexadecimal_prefix')
    '''
    if USE_RE:
        return read(s,pos, regexp(r'0[xX]',errproc='hexadecimal_prefix'))
    else:
        return read_oneof(s,pos,'0x','0X',errproc='hexadecimal_prefix')

@cacheread
def hex_quad(s,pos): # without universal character name
    '''
    >>> ptest(hex_quad,'0134')
    308
    >>> ptest(hex_quad,'013')
    Err(0, 'hex_quad')
    '''
    if USE_RE:
        return read(s,pos, regexp(r'[0-9a-zA-Z]{4}',errproc='hex_quad',proc = lambda x : int(x[0],16)))
    else:
        return read_sequential(s,pos,a=hexadecimal_digit,b=hexadecimal_digit,c=hexadecimal_digit,d=hexadecimal_digit,
                         proc = dcatf(inthex), errproc = 'hex_quad' ) # almost dcat
                              # lambda d: int(d.a+d.b+d.c+d.d,16)

@cacheread
def universal_character_name(s,pos):
    r'''
    >>> ptest(universal_character_name,r"\u0130")
    'İ'
    >>> ptest(universal_character_name,r"\U000103a6")
    '𐎦'
    >>> ptest(universal_character_name,'0134')
    Err(0, 'universal_character_name')
    '''
    if USE_RE:
        return read(s,pos, regexp(r'\\u[0-9a-zA-Z]{4}|\\U[0-9a-zA-Z]{8}',errproc='universal_character_name',
                                  proc=lambda x : chr(int(x[0][2:],16))))
    else:
        return read_oneof(s,pos,sequential(a='\\u',b=hex_quad, proc= lambda d: chr(d.b) , errproc= '\\uhhhh' ),
                sequential(a='\\U',b=hex_quad,c=hex_quad, proc= lambda d: chr(d.b*65536+d.c) , errproc= '\\Uhhhhhhhh' ),
                         errproc='universal_character_name')

@cacheread
def identifier(s,pos):
    '''
    >>> ptest(identifier,'as0df')
    'as0df'
    >>> ptest(identifier,'0as0df')
    Err(0, 'identifier')
    '''
    if USE_RE:
        return read(s,pos, regexp(r'[a-zA-Z_][a-zA_Z0-9_]*',errproc='identifier')) # without universal character name ##todo
    else:
        return read_sequential(s,pos, f=oneof(nondigit,universal_character_name), 
                                      s=repeatedly(0,infinity,oneof(nondigit,universal_character_name,digit), proc=lcat ),
                              proc=dcat, errproc= 'identifier'
                              )


In [12]:
nonzero_digit = char_in_set('123456789',errproc='nonzero_digit')

@cacheread
def decimal_constant(s,pos):
    '''
    >>> ptest(decimal_constant,'123')
    123
    >>> ptest(decimal_constant,'0123')
    Err(0, 'decimal_constant')
    '''
    if USE_RE:
        return read(s,pos, regexp(r'[1-9][0-9]*',proc = lambda x : int(x[0]), errproc='decimal_constant'))
    else:
        return read_sequential(s,pos, f=nonzero_digit, s=repeatedly(0,infinity,digit, proc=lcat) ,
                              proc=dcatf(int), errproc= 'decimal_constant'
                              )

@cacheread
def octal_constant(s,pos):
    '''
    >>> ptest(octal_constant,'0123')
    83
    >>> ptest(octal_constant,'0')
    0
    >>> ptest(octal_constant,'123')
    Err(0, 'octal_constant')
    '''
    if USE_RE:
        return read(s,pos, regexp(r'0[0-7]*',proc = lambda x : int(x[0],8), errproc='octal_constant'))
    else:
        return read_sequential(s,pos, f='0', s=rep_cat(0,infinity,octal_digit) ,
                              proc=lambda d: int(d.s,8) if len(d.s) else 0, errproc= 'octal_constant' #
                              )

@cacheread
def hexadecimal_digit_sequence(s,pos):
    '''
    >>> ptest(hexadecimal_digit_sequence,'123')
    mkdict(v=291,l=3)
    >>> ptest(hexadecimal_digit_sequence,'00123')
    mkdict(v=291,l=5)
    >>> ptest(hexadecimal_digit_sequence,'-123')
    Err(0, 'hexadecimal_digit_sequence')
    '''
    def foo(l):
        s = ''.join(l)
        return mkdict(v=inthex(s),l=len(s))
    if USE_RE:
        return read(s,pos, regexp(r'[0-9a-fA-F]+',proc = lambda x : mkdict(v=int(x[0],16),l=len(x[0])), errproc='hexadecimal_digit_sequence'))
    else:
        return read_repeatedly(s,pos, 1,infinity,hexadecimal_digit, proc=foo, errproc= 'hexadecimal_digit_sequence')

@cacheread
def hexadecimal_constant(s,pos):
    '''
    >>> ptest(hexadecimal_constant,'0x123')
    291
    >>> ptest(hexadecimal_constant,'0x-123')
    Err(0, 'hexadecimal_constant')
    >>> ptest(hexadecimal_constant,'123')
    Err(0, 'hexadecimal_constant')
    '''
    if USE_RE:
        return read(s,pos, regexp(r'0[xX][0-9a-fA-F]+',proc = lambda x : int(x[0],16), errproc='hexadecimal_constant'))
    else:
        return read_sequential(s,pos, f=oneof('0x','0X'), s=hexadecimal_digit_sequence ,
                              proc=lambda d: d.s.v, errproc= 'hexadecimal_constant'
                              )


unsigned_suffix = char_in_set('uU',errproc='unsigned_suffix')
long_suffix     = char_in_set('lL',errproc='long_suffix')

@cacheread
def long_long_suffix(s,p):
    if USE_RE:
        return read(s,p, regexp(r'll|LL', errproc='long_long_suffix'))
    else:
        return read_oneof(s,p, 'll', 'LL', errproc='long_long_suffix')

@cacheread
def integer_suffix(s,p):
    '''
    >>> ptest(integer_suffix,'u')
    mkdict(unsigned=True,long=0)
    >>> ptest(integer_suffix,'uLL')
    mkdict(unsigned=True,long=2)
    '''
    def final_proc(s):
        d = AttrDict()
        d.unsigned =  'u' in s or 'U' in s
        d.long = 2 if 'll' in s or 'LL' in s else \
                1 if 'l' in s or 'L' in s else \
                0
        return d
    def proc_first(proc):
        def f(p):
            a,b = p
            a = proc(a)
            return (a,b)
        return f
        
    if USE_RE:
        return read(s,p, regexp(r'[uU](ll|LL|[lL])?|[lL][uU]?|(ll|LL)[uU]?', errproc='integer_suffix',
                               proc=lambda x : final_proc(x[0])))
    else:
        lL = char_in_set('lL')
        uU = char_in_set('uU')
        return read_atleast_oneof(s,p, seq_cat(a=uU,b=opt_des(atleast_oneof('ll','LL',lL,proc=select_longest))),
                                seq_cat(a=lL,b=opt_des(uU)),
                                seq_cat(a=oneof('ll','LL'),b=opt_des(uU)),
                          proc=compose(proc_first(final_proc),select_longest),errproc='integer_suffix')
@cacheread
def integer_constant(s,p):
    '''
    >>> ptest(integer_constant,'5')
    mkdict(unsigned=False,long=0,value=5)
    >>> ptest(integer_constant,'0x5u')
    mkdict(unsigned=True,long=0,value=5)
    >>> ptest(integer_constant,'0')
    mkdict(unsigned=False,long=0,value=0)
    '''
    return read_sequential(s,p, i=atleast_oneof(decimal_constant,octal_constant,hexadecimal_constant,proc=select_longest), 
                                s=optional(integer_suffix,proc=dflt(mkdict(unsigned=False,long=0))),
                         proc=lambda d: dict_append(d.s,value=d.i),errproc='integer_constant')
    


In [13]:
floating_suffix = char_in_set('fFlL',proc=lambda c : 'float' if c in 'fF' else 'long_double',errproc='char_in_set')
sign = char_in_set('+-',errproc='sign')
@cacheread
def fractional_constant(s,p):
    '''
    >>> ptest(fractional_constant,'2.')
    '2.'
    >>> ptest(fractional_constant,'.') 
    Err(0, 'fractional_constant')
    '''
    if USE_RE:
        return read(s,p,regexp(r'[0-9]*\.[0-9]+|[0-9]+\.',errproc='fractional_constant'))
    else:
        return read_oneof(s,p, seq_cat(a=rep_cat(1,infinity,digit),b='.',c=rep_cat(0,infinity,digit)),
                             seq_cat(b='.',c=rep_cat(1,infinity,digit)), errproc='fractional_constant')
@cacheread
def exponent_part(s,p):
    '''
    >>> ptest(exponent_part,'e76')
    'e76'
    >>> ptest(exponent_part,'E-0')
    'E-0'
    >>> ptest(exponent_part,'.')
    Err(0, 'exponent_part')
    '''
    if USE_RE:
        return read(s,p,regexp(r'[eE][+-]?[0-9]+',errproc='exponent_part'))
    else:
        return read(s,p, seq_cat(a=char_in_set('eE'),b=opt_des(char_in_set('+-')),c=rep_cat(1,infinity,digit), errproc='exponent_part'))
@cacheread
def decimal_floating_constant(s,p): # ignore long double ##todo
    '''
    >>> ptest(decimal_floating_constant,'5e2')
    mkodict(value=500.0,type='double')
    >>> ptest(decimal_floating_constant,'0.f')
    mkodict(value=0.0,type='float')
    >>> ptest(decimal_floating_constant,'0.e') 
    Err(2, ' ')
    >>> ptest(decimal_floating_constant,'1e-5L')
    mkodict(value=1e-05,type='long_double')
    >>> ptest(decimal_floating_constant,'1')
    Err(0, 'decimal_floating_constant')
    '''
    if USE_RE:
        return read_sequence(s,p,value=regexp(r'[0-9]*\.[0-9]+|[0-9]+\.([eE][+-]?[0-9]+)?|[0-9]+[eE][+-]?[0-9]+',proc= lambda x: float(x[0])),
                             type=optional(floating_suffix,proc=dflt('double')), errproc='decimal_floating_constant')
    else:
        return read_sequence(s,p,value=oneof(seq_cat(a=fractional_constant,b=opt_des(exponent_part)),
                                             seq_cat(a=rep_cat(1,infinity,digit),b=exponent_part),
                                             proc= lambda x: float(x)),
                             type=optional(floating_suffix,proc=dflt('double')), errproc='decimal_floating_constant')
@cacheread
def my_hexadecimal_fractional_constant(s,p):
    r'''
    hs(\.hs?)?|\.hs
    >>> ptest(my_hexadecimal_fractional_constant,'2')
    2.0
    >>> ptest(my_hexadecimal_fractional_constant,'2.')
    2.0
    >>> ptest(my_hexadecimal_fractional_constant,'.2')
    0.125
    >>> ptest(my_hexadecimal_fractional_constant,'.')
    Err(0, 'my_hexadecimal_fractional_constant')
    '''
    hs = hexadecimal_digit_sequence
    return read_oneof(s,p, sequential(a=hs,
                                      b=optional(sequential(a='.',b=optional(hs,proc=dflt(mkdict(v=0,l=0))),proc=lambda d: d.b.v/16**d.b.l),proc=dflt(0.)),
                                      proc= lambda d:d.a.v+d.b),
                     sequential(a='.',b=hs,proc=lambda d: d.b.v/16**d.b.l),errproc='my_hexadecimal_fractional_constant')
@cacheread
def binary_exponent_part(s,p):
    '''
    >>> ptest(binary_exponent_part,'p76')
    76
    >>> ptest(binary_exponent_part,'P-0')
    0
    >>> ptest(binary_exponent_part,'.')
    Err(0, 'binary_exponent_part')
    '''
    if USE_RE:
        return read(s,p,regexp(r'[pP][+-]?[0-9]+',proc= lambda x  : int(x[0][1:]),errproc='binary_exponent_part'))
    else:
        return read_sequential(s,p, a=char_in_set('pP'),b=optional(char_in_set('+-'),proc=dflt('+')),c=rep_cat(1,infinity,digit),
                          proc=lambda d : int(d.c) if d.b=='+' else -int(d.c),
                          errproc='binary_exponent_part')
@cacheread
def hexadecimal_floating_constant(s,p): # ignore long double ##todo
    '''
    >>> ptest(hexadecimal_floating_constant,'0x5p2')
    mkdict(value=20.0,type='double')
    >>> ptest(hexadecimal_floating_constant,'0x0.p0f')
    mkdict(value=0.0,type='float')
    >>> ptest(hexadecimal_floating_constant,'0x0.p') 
    Err(0, 'hexadecimal_floating_constant')
    >>> ptest(hexadecimal_floating_constant,'0x1p-5L')
    mkdict(value=0.03125,type='long_double')
    >>> ptest(hexadecimal_floating_constant,'0x1')
    Err(0, 'hexadecimal_floating_constant')
    '''
    return read_sequence(s,p,pref=regexp('0x|0X'),value=my_hexadecimal_fractional_constant,exp=binary_exponent_part,
                         type=optional(floating_suffix,proc=dflt('double')), 
                         proc=lambda d : mkdict(value=d.value*2**d.exp,type=d.type), 
                         errproc='hexadecimal_floating_constant')
def floating_constant(s,p):
    return read_oneof(s,p,hexadecimal_floating_constant,decimal_floating_constant)

doctest.testmod()

TestResults(failed=0, attempted=47)

In [14]:
pos = mkpos(0)
print(hexadecimal_floating_constant('0x5p2 ',pos))
print(pos)
print(repr(read('0x5p2 ',pos,fix_str(' '))))
pos

mkdict(value=20.0,type='double')
mkdict(x=5)
' '


mkdict(x=6)

In [15]:
ptest(hexadecimal_floating_constant,'0x5p2')

mkdict(value=20.0,type='double')

In [16]:
USE_RE=False ; read_sequence('0x5p2 ',mkpos(0), a=hexadecimal_floating_constant,b=' ')

mkodict(a=mkdict(value=20.0,type='double'),b=' ')

In [17]:
USE_RE=True ; hexadecimal_constant('0x123',mkpos(0)) # doctest:

291

In [18]:
USE_RE = True

In [19]:
hexadecimal_prefix('0x000',mkpos(0))

'0x'

In [20]:
hexadecimal_prefix('0000',mkpos(0)) # doctest: +ELLIPSIS

parseError([(0, 'hexadecimal_prefix')], None)

In [21]:
hex_quad('0134',mkpos(0))

308

In [22]:
hex_quad('013',mkpos(0))

parseError([(0, 'hex_quad')], None)

In [23]:
universal_character_name('\\u0130',mkpos(0))

'İ'

In [24]:
universal_character_name('\\U000103a6',mkpos(0))

'𐎦'

In [25]:
universal_character_name('0134',mkpos(0))

parseError([(0, 'universal_character_name')], None)

In [26]:
identifier('as0d f',mkpos(0))

'as0d'

In [27]:
identifier('0as0df',mkpos(0))

parseError([(0, 'identifier')], None)

In [28]:
parseError([(0, 'one of')], [parseError([(0, '0x')], None), parseError([(0, '0X')], None)])

parseError([(0, 'one of')], [parseError([(0, '0x')], None), parseError([(0, '0X')], None)])

In [29]:
def gen_fun(name):
    def default_name():
        return 1/0
    default_name.__name__ = name
    return default_name
foo = gen_fun('hello')
pass
pass
pass
foo()

ZeroDivisionError: division by zero